<a href="https://colab.research.google.com/github/vivandsouza/IIIT-Madram-Sangam-Hackathon/blob/master/Sangam2019_best_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from google.colab import files
import requests

def download_file_from_google_drive(id, destination):
   URL = "https://docs.google.com/uc?export=download"

   session = requests.Session()

   response = session.get(URL, params = { 'id' : id }, stream = True)
   token = get_confirm_token(response)

   if token:
           params = { 'id' : id, 'confirm' : token }
           response = session.get(URL, params = params, stream = True)

   save_response_content(response, destination)

def get_confirm_token(response):
   for key, value in response.cookies.items():
           if key.startswith('download_warning'):
               return value

   return None

def save_response_content(response, destination):
   CHUNK_SIZE = 32768

   with open(destination, "wb") as f:
     for chunk in response.iter_content(CHUNK_SIZE):
       if chunk: # filter out keep-alive new chunks
         f.write(chunk)
file_id = '1df7g1HOR0sCKnUxmee270Z-DH_MS6eyb'
destination = 'Train.csv'
download_file_from_google_drive(file_id, destination)

file_id = '1uvA4ta4yiREvD58MpkwwWkLoyzt1wa2M'
destination = 'Test.csv'

download_file_from_google_drive(file_id, destination)

In [0]:
import pandas as pd
import numpy as np

In [0]:
train_data = pd.read_csv("Train.csv")
test_data = pd.read_csv("Test.csv")

In [0]:
print(train_data.shape)
print(test_data.shape)

(33750, 15)
(14454, 14)


In [0]:
week = test_data["date_time"]

In [0]:
train_data["date_time"] = pd.to_datetime(train_data["date_time"], format = "%Y%m%d %H:%M:%S")
train_data["date_time"] = train_data["date_time"].dt.day_name()

test_data["date_time"] = pd.to_datetime(test_data["date_time"], format = "%Y%m%d %H:%M:%S")
test_data["date_time"] = test_data["date_time"].dt.day_name()

In [0]:
train_data["date_time"] = train_data["date_time"].astype("category")
train_data["is_holiday"] = train_data["is_holiday"].astype("category")
train_data["weather_type"] = train_data["weather_type"].astype("category")
train_data["weather_description"] = train_data["weather_description"].astype("category")

test_data["date_time"] = test_data["date_time"].astype("category")
test_data["is_holiday"] = test_data["is_holiday"].astype("category")
test_data["weather_type"] = test_data["weather_type"].astype("category")
test_data["weather_description"] = test_data["weather_description"].astype("category")

In [0]:
## Splitting numerical and categorical variables
train_data_num = train_data.iloc[:, 2:12]
test_data_num = test_data.iloc[:, 2:12]

In [0]:
## Encoding categorical data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [0]:
cat_var = ["date_time", "is_holiday", "weather_type", "weather_description"]
train_data_cat = pd.DataFrame()
test_data_cat = pd.DataFrame()
for col in cat_var:
    train_data_cat[col] = le.fit_transform(train_data[col])
    test_data_cat[col] = le.transform(test_data[col])

In [0]:
print(train_data.shape, train_data_cat.shape)
print(test_data.shape, test_data_cat.shape)

(33750, 15) (33750, 4)
(14454, 14) (14454, 4)


In [0]:
for i in ["date_time", "is_holiday", "weather_type", "weather_description"]:
    print(np.unique(train_data[i]),"\n", np.unique(train_data_cat[i]))

['Friday' 'Monday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday' 'Wednesday'] 
 [0 1 2 3 4 5 6]
['Christmas Day' 'Columbus Day' 'Independence Day' 'Labor Day'
 'Martin Luther King Jr Day' 'Memorial Day' 'New Years Day' 'None'
 'State Fair' 'Thanksgiving Day' 'Veterans Day' 'Washingtons Birthday'] 
 [ 0  1  2  3  4  5  6  7  8  9 10 11]
['Clear' 'Clouds' 'Drizzle' 'Fog' 'Haze' 'Mist' 'Rain' 'Smoke' 'Snow'
 'Squall' 'Thunderstorm'] 
 [ 0  1  2  3  4  5  6  7  8  9 10]
['SQUALLS' 'Sky is Clear' 'broken clouds' 'drizzle' 'few clouds' 'fog'
 'freezing rain' 'haze' 'heavy intensity drizzle' 'heavy intensity rain'
 'heavy snow' 'light intensity drizzle' 'light intensity shower rain'
 'light rain' 'light rain and snow' 'light shower snow' 'light snow'
 'mist' 'moderate rain' 'overcast clouds' 'proximity shower rain'
 'proximity thunderstorm' 'proximity thunderstorm with drizzle'
 'proximity thunderstorm with rain' 'scattered clouds' 'shower drizzle'
 'shower snow' 'sky is clear' 'sleet' 'smoke' 'sn

In [0]:
for i in ["date_time", "is_holiday", "weather_type", "weather_description"]:
    print(np.unique(test_data[i]),"\n", np.unique(test_data_cat[i]))

['Friday' 'Monday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday' 'Wednesday'] 
 [0 1 2 3 4 5 6]
['Christmas Day' 'Columbus Day' 'Independence Day' 'Labor Day'
 'Martin Luther King Jr Day' 'Memorial Day' 'New Years Day' 'None'
 'State Fair' 'Thanksgiving Day' 'Veterans Day' 'Washingtons Birthday'] 
 [ 0  1  2  3  4  5  6  7  8  9 10 11]
['Clear' 'Clouds' 'Drizzle' 'Fog' 'Haze' 'Mist' 'Rain' 'Smoke' 'Snow'
 'Thunderstorm'] 
 [ 0  1  2  3  4  5  6  7  8 10]
['Sky is Clear' 'broken clouds' 'drizzle' 'few clouds' 'fog' 'haze'
 'heavy intensity drizzle' 'heavy intensity rain' 'heavy snow'
 'light intensity drizzle' 'light intensity shower rain' 'light rain'
 'light shower snow' 'light snow' 'mist' 'moderate rain' 'overcast clouds'
 'proximity shower rain' 'proximity thunderstorm'
 'proximity thunderstorm with drizzle' 'proximity thunderstorm with rain'
 'scattered clouds' 'shower drizzle' 'sky is clear' 'sleet' 'smoke' 'snow'
 'thunderstorm' 'thunderstorm with heavy rain'
 'thunderstorm with ligh

In [0]:
print(train_data_num.shape)
print(train_data_cat.shape)

(33750, 10)
(33750, 4)


In [0]:
## Add numerical and categorical data
train = pd.concat([train_data_num, train_data_cat], axis = 1)
test = pd.concat([test_data_num, test_data_cat], axis = 1)

In [0]:
print(train.shape)
print(test.shape)

(33750, 14)
(14454, 14)


In [0]:
print(max(train["rain_p_h"]))
print(train.loc[train["rain_p_h"] == 9831.3].index)
# ## Drop that row in train data
train = train.drop(train.index[24872])

9831.3
Int64Index([24872], dtype='int64')


In [0]:
X = train
y = train_data["traffic_volume"]

In [0]:
# ## Drop the same index 24872 in target also
y = y.drop(y.index[24872])
y = y.values

In [0]:
print(X.shape)
print(y.shape)

(33749, 14)
(33749,)


In [0]:
## Splitting the dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [0]:
y_train

array([ 826, 4894, 5882, ...,  529, 5858,  248])

In [0]:
## Feature scaling (Standardization)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
test = sc.transform(test)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Add
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.layers import Activation

Using TensorFlow backend.


In [0]:
NN_model = Sequential()
# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = 14, activation='relu'))
# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

W0802 11:55:52.380027 139951705397120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0802 11:55:52.386497 139951705397120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 11:55:52.395265 139951705397120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.



In [0]:
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

W0802 11:55:52.477131 139951705397120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1920      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 166,785
Trainable params: 166,785
Non-trainable params: 0
_________________________________________________________________


In [0]:
NN_model.fit(X_train, y_train, batch_size=32, nb_epoch=20, verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.
W0802 11:55:52.699289 139951705397120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0802 11:55:52.869379 139951705397120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/20
26999/26999 [==============================] - 5s 199us/step - loss: 1782.8255 - mean_absolute_error: 1782.8255
Epoch 2/20
26999/26999 [==============================] - 5s 168us/step - loss: 1673.9560 - mean_absolute_error: 1673.9560
Epoch 3/20
26999/26999 [==============================] - 4s 167us/step - loss: 1662.2850 - mean_absolute_error: 1662.2850
Epoch 4/20
26999/26999 [==============================] - 4s 161us/step - loss: 1660.0239 - mean_absolute_error: 1660.0239
Epoch 5/20
26999/26999 [==============================] - 4s 162us/step - loss: 1654.0824 - mean_absolute_error: 1654.0824
Epoch 6/20
26999/26999 [==============================] - 4s 161us/step - loss: 1650.9347 - mean_absolute_error: 1650.9347
Epoch 7/20
26999/26999 [==============================] - 4s 162us/step - loss: 1647.9628 - mean_absolute_error: 1647.9628
Epoch 8/20
26999/26999 [==============================] - 4s 162us/step - loss: 1644.9591 - mean_absolute_error: 1644.9591
Epoch 9/20
26999

In [0]:
test_pred = NN_model.predict(test)

In [0]:
test_pred = test_pred.astype("int32")

In [0]:
temp_list = []
for i in range(len(test_pred)):
  for j in range(len(test_pred[i])):
    temp_list.append(test_pred[i][j])

In [0]:
final = pd.DataFrame(week)
final["traffic_volume"] = temp_list

In [0]:
final.to_csv("submission_9.csv", index = False)